## Gradient Boosting Tree Regression

Unlike in Random Forests where decision trees act independently, and a major vote of the trees is taken as the output, Gradient Boosting Trees (GBT) iteratively aggregates decision trees to improve on each other's error. You can imagine GBT as a chain of decision trees, and the initial prediction is fed into a subsequent model, propagating all the way to the final model in the chain.

The algorithm is called 'Gradient Boosting' because trees are made to boost the collective model's performance by following the negative gradient of a loss function. Taking a gradient descent step becomes the same as fitting a tree model to the residual (difference between the observed value and model output). 

    `Sum of Squared Error = (observation - prediction)^2
    `derivative of SSE with respect to prediction = 2 * (observation - prediction) * -1 = -2 * (observation - prediction) = -2 * residual

We are optimizing using gradient descent, but our step does not involve manipulating parameters since trees are non-parametric. We optimize in the functional space, not the parametric space, i.e the tree's internals are not considered and instead just its output. This type of gradient descent is called **Functional Gradient Descent**.

In [342]:
import numpy as np
import pandas as pd
from sklearn.ensemble import GradientBoostingRegressor as GBR
from sklearn.datasets import load_diabetes, fetch_california_housing
from sklearn.tree import DecisionTreeRegressor as DTR
import os

In [343]:
def mean_squared_error(y_true, y_hat):

    return 1 / len(y_true) * np.sum((y_true - y_hat)**2)

def root_mean_squared_error(y_true, y_hat):

    return (1 / len(y_true) * np.sum((y_true - y_hat)**2)) ** 1/2

In [ ]:
class Node:

    def __init__(self, left, right, feature, split, sse, avg_target):

        self.left = left
        self.right = right
        self.feature = feature
        self.split = split
        self.sse = sse
        self.avg_target = avg_target

class DecisionTreeRegressor:

    def __init__(self, max_depth, min_samples_split, max_features):
        
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.max_features = max_features
        self.root = None
    
    def get_sse(self, data, target_col):

        return np.sum((data[target_col] - data[target_col].mean()) ** 2)

    def fit(self, data, target_col):

        if len(data) == 0 or target_col not in data.columns:

            print("data_copy is empty or target col not in data")

            return
    
        self.root = self._fit(data, target_col, float("inf"), 0)
    
    def _fit(self, data, target_col, parent_sse, depth):

        if len(data) < self.min_samples_split or depth >= self.max_depth:

            return Node(None, None, None, None, self.get_sse(data, target_col), data[target_col].mean())

        features = [f for f in data.columns if f != target_col]

        best_sse, best_split = parent_sse, 0
        best_feat = None

        for feat in features:

            unique_sorted = sorted(np.unique(data[feat]))

            for i in range(1, len(unique_sorted)):

                split = (unique_sorted[i] + unique_sorted[i - 1]) / 2

                left_region = data[data[feat] < split]
                right_region = data[data[feat] >= split]

                total_sse = self.get_sse(left_region, target_col) + self.get_sse(right_region, target_col)

                if total_sse < best_sse:

                    best_sse = total_sse
                    best_split = split
                    best_feat = feat
        
        if not best_feat:

            return Node(None, None, None, None, self.get_sse(data, target_col), data[target_col].mean())

        node = Node(None, None, best_feat, best_split, best_sse, None)

        node.left = self._fit(data[data[best_feat] < best_split], target_col, best_sse, depth+1)
        node.right = self._fit(data[data[best_feat] >= best_split], target_col, best_sse, depth+1)
        return node
    
    def predict(self, data):

        if len(data) == 0:

            print("Data is empty")

            return
        
        return np.array([self._predict(row[1]) for row in data.iterrows()])

    def _predict(self, row):

        curr = self.root

        while curr.left and curr.right:

            feat = curr.feature
            split = curr.split

            if row[feat] < split:

                curr = curr.left
            
            else:

                curr = curr.right
        
        return curr.avg_target
    

class GradientBoostingTreeRegression:

    def __init__(self, num_estimators, max_depth, min_samples_split, max_features, learning_rate):
        
        self.num_estimators = num_estimators
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.max_features = max_features
        self.learning_rate = learning_rate
        self.ensemble = {}
        self.initial_prediction = None
        self.ensemble_order = []
    
    def fit(self, data, target_col):

        if len(data) == 0 or target_col not in data.columns:

            print("Data is empty or target not in columns")

            return

        self._fit(data, target_col)
    
    def _fit(self, data, target_col):

        data_copy = data.copy(deep=True)

        original_target_col = data_copy[target_col].copy()
        data_copy.drop(columns=[target_col], inplace=True)

        self.initial_prediction = original_target_col.mean()

        data_copy["residual"] = original_target_col - self.initial_prediction

        cumult_pred = np.full_like(original_target_col, fill_value=self.initial_prediction)

        for i in range(self.num_estimators):

            stump = DecisionTreeRegressor(self.max_depth, self.min_samples_split, self.max_features)

            stump.fit(data_copy, "residual")

            features_only = data_copy.drop(columns=["residual"])

            stump_pred = stump.predict(features_only)

            self.ensemble_order.append(stump)

            gradient = stump_pred # based on MSE. the constant from the derivative we scale down. doesnt affect location of minimum

            cumult_pred = cumult_pred + self.learning_rate*gradient

            self.ensemble[f"decision_tree_{i+1}"] = {"MSE":mean_squared_error(original_target_col, cumult_pred), "RMSE": root_mean_squared_error(original_target_col, cumult_pred), "Fitted Model": stump}

            data_copy["residual"] = data_copy["residual"] - self.learning_rate*gradient

    
    def predict(self,data):

        cumult_pred = self.initial_prediction

        for tree in self.ensemble_order:

            cumult_pred += self.learning_rate*tree.predict(data)
        
        return cumult_pred







            

        


In [345]:
# housing = load_diabetes(as_frame=True)


red_wine = pd.read_csv(os.path.join("datasets", "winequality-red.csv"))

data = red_wine
target_col = "quality"

print(data)
print(data.shape)

      fixed acidity  volatile acidity  citric acid  ...  sulphates  alcohol  quality
0               7.4             0.700         0.00  ...       0.56      9.4        5
1               7.8             0.880         0.00  ...       0.68      9.8        5
2               7.8             0.760         0.04  ...       0.65      9.8        5
3              11.2             0.280         0.56  ...       0.58      9.8        6
4               7.4             0.700         0.00  ...       0.56      9.4        5
...             ...               ...          ...  ...        ...      ...      ...
1594            6.2             0.600         0.08  ...       0.58     10.5        5
1595            5.9             0.550         0.10  ...       0.76     11.2        6
1596            6.3             0.510         0.13  ...       0.75     11.0        6
1597            5.9             0.645         0.12  ...       0.71     10.2        5
1598            6.0             0.310         0.47  ...       0.6

In [346]:
train_size = 0.8
train_data = data[:int(len(data)*train_size)]
test_data = data[int(len(data)*train_size):]

print(train_data.shape)
print(test_data.shape)

(1279, 12)
(320, 12)


In [347]:
max_depth, min_samples_split, max_features = 5, 5, len(train_data.columns)-1

my_dtr = DecisionTreeRegressor(max_depth, min_samples_split, max_features)
sklearn_dtr = DTR(max_depth=max_depth, max_features=max_features, min_samples_split=min_samples_split)

my_dtr.fit(train_data, target_col)
sklearn_dtr.fit(train_data.drop(columns=[target_col]), train_data[target_col])

,"criterion criterion: {""squared_error"", ""friedman_mse"", ""absolute_error"", ""poisson""}, default=""squared_error""The function to measure the quality of a split. Supported criteriaare ""squared_error"" for the mean squared error, which is equal tovariance reduction as feature selection criterion and minimizes the L2loss using the mean of each terminal node, ""friedman_mse"", which usesmean squared error with Friedman's improvement score for potentialsplits, ""absolute_error"" for the mean absolute error, which minimizesthe L1 loss using the median of each terminal node, and ""poisson"" whichuses reduction in the half mean Poisson deviance to find splits... versionadded:: 0.18 Mean Absolute Error (MAE) criterion... versionadded:: 0.24 Poisson deviance criterion.",'squared_error'
,"splitter splitter: {""best"", ""random""}, default=""best""The strategy used to choose the split at each node. Supportedstrategies are ""best"" to choose the best split and ""random"" to choosethe best random split.",'best'
,"max_depth max_depth: int, default=NoneThe maximum depth of the tree. If None, then nodes are expanded untilall leaves are pure or until all leaves contain less thanmin_samples_split samples.For an example of how ``max_depth`` influences the model, see:ref:`sphx_glr_auto_examples_tree_plot_tree_regression.py`.",5
,"min_samples_split min_samples_split: int or float, default=2The minimum number of samples required to split an internal node:- If int, then consider `min_samples_split` as the minimum number.- If float, then `min_samples_split` is a fraction and `ceil(min_samples_split * n_samples)` are the minimum number of samples for each split... versionchanged:: 0.18 Added float values for fractions.",5
,"min_samples_leaf min_samples_leaf: int or float, default=1The minimum number of samples required to be at a leaf node.A split point at any depth will only be considered if it leaves atleast ``min_samples_leaf`` training samples in each of the left andright branches. This may have the effect of smoothing the model,especially in regression.- If int, then consider `min_samples_leaf` as the minimum number.- If float, then `min_samples_leaf` is a fraction and `ceil(min_samples_leaf * n_samples)` are the minimum number of samples for each node... versionchanged:: 0.18 Added float values for fractions.",1
,"min_weight_fraction_leaf min_weight_fraction_leaf: float, default=0.0The minimum weighted fraction of the sum total of weights (of allthe input samples) required to be at a leaf node. Samples haveequal weight when sample_weight is not provided.",0.0
,"max_features max_features: int, float or {""sqrt"", ""log2""}, default=NoneThe number of features to consider when looking for the best split:- If int, then consider `max_features` features at each split.- If float, then `max_features` is a fraction and `max(1, int(max_features * n_features_in_))` features are considered at each split.- If ""sqrt"", then `max_features=sqrt(n_features)`.- If ""log2"", then `max_features=log2(n_features)`.- If None, then `max_features=n_features`.Note: the search for a split does not stop until at least onevalid partition of the node samples is found, even if it requires toeffectively inspect more than ``max_features`` features.",11
,"random_state random_state: int, RandomState instance or None, default=NoneControls the randomness of the estimator. The features are alwaysrandomly permuted at each split, even if ``splitter`` is set to``""best""``. When ``max_features < n_features``, the algorithm willselect ``max_features`` at random at each split before finding the bestsplit among them. But the best found split may vary across differentruns, even if ``max_features=n_features``. That is the case, if theimprovement of the criterion is identical for several splits and onesplit has to be selected at random. To obtain a deterministic behaviourduring fitting, ``random_state`` has to be fixed to an integer.See :term:`Glossary ` for details.",None
,"max_leaf

In [348]:
my_dtr_preds = my_dtr.predict(test_data)
sklearn_preds = sklearn_dtr.predict(test_data.drop(columns=[target_col]))

pd.DataFrame({"Mean Squared Error": [mean_squared_error(test_data[target_col], my_dtr_preds), mean_squared_error(test_data[target_col], sklearn_preds)],
              "Root Mean Squared Error": [root_mean_squared_error(test_data[target_col], my_dtr_preds), root_mean_squared_error(test_data[target_col], sklearn_preds)]}, index=["My DTR", "SKlearn DTR"])

,Mean Squared Error,Root Mean Squared Error
My DTR,0.493988,0.246994
SKlearn DTR,0.484884,0.242442


In [349]:
num_estimators, max_depth, min_samples_split, max_features, learning_rate = 10, 3, 3, len(train_data.columns)-1, 0.1

my_gbtr = GradientBoostingTreeRegression(num_estimators, max_depth, min_samples_split, max_features, learning_rate)
my_gbtr.fit(train_data, target_col)

sklearn_gbtr = GBR(n_estimators=num_estimators, max_depth=max_depth, min_samples_split=min_samples_split, max_features=max_features, learning_rate=learning_rate)
sklearn_gbtr.fit(train_data.drop(columns=[target_col]), train_data[target_col])

,"loss loss: {'squared_error', 'absolute_error', 'huber', 'quantile'}, default='squared_error'Loss function to be optimized. 'squared_error' refers to the squarederror for regression. 'absolute_error' refers to the absolute error ofregression and is a robust loss function. 'huber' is acombination of the two. 'quantile' allows quantile regression (use`alpha` to specify the quantile).See:ref:`sphx_glr_auto_examples_ensemble_plot_gradient_boosting_quantile.py`for an example that demonstrates quantile regression for creatingprediction intervals with `loss='quantile'`.",'squared_error'
,"learning_rate learning_rate: float, default=0.1Learning rate shrinks the contribution of each tree by `learning_rate`.There is a trade-off between learning_rate and n_estimators.Values must be in the range `[0.0, inf)`.",0.1
,"n_estimators n_estimators: int, default=100The number of boosting stages to perform. Gradient boostingis fairly robust to over-fitting so a large number usuallyresults in better performance.Values must be in the range `[1, inf)`.",10
,"subsample subsample: float, default=1.0The fraction of samples to be used for fitting the individual baselearners. If smaller than 1.0 this results in Stochastic GradientBoosting. `subsample` interacts with the parameter `n_estimators`.Choosing `subsample < 1.0` leads to a reduction of varianceand an increase in bias.Values must be in the range `(0.0, 1.0]`.",1.0
,"criterion criterion: {'friedman_mse', 'squared_error'}, default='friedman_mse'The function to measure the quality of a split. Supported criteria are""friedman_mse"" for the mean squared error with improvement score byFriedman, ""squared_error"" for mean squared error. The default value of""friedman_mse"" is generally the best as it can provide a betterapproximation in some cases... versionadded:: 0.18",'friedman_mse'
,"min_samples_split min_samples_split: int or float, default=2The minimum number of samples required to split an internal node:- If int, values must be in the range `[2, inf)`.- If float, values must be in the range `(0.0, 1.0]` and `min_samples_split` will be `ceil(min_samples_split * n_samples)`... versionchanged:: 0.18 Added float values for fractions.",3
,"min_samples_leaf min_samples_leaf: int or float, default=1The minimum number of samples required to be at a leaf node.A split point at any depth will only be considered if it leaves atleast ``min_samples_leaf`` training samples in each of the left andright branches. This may have the effect of smoothing the model,especially in regression.- If int, values must be in the range `[1, inf)`.- If float, values must be in the range `(0.0, 1.0)` and `min_samples_leaf` will be `ceil(min_samples_leaf * n_samples)`... versionchanged:: 0.18 Added float values for fractions.",1
,"min_weight_fraction_leaf min_weight_fraction_leaf: float, default=0.0The minimum weighted fraction of the sum total of weights (of allthe input samples) required to be at a leaf node. Samples haveequal weight when sample_weight is not provided.Values must be in the range `[0.0, 0.5]`.",0.0
,"max_depth max_depth: int or None, default=3Maximum depth of the individual regression estimators. The maximumdepth limits the number of nodes in the tree. Tune this parameterfor best performance; the best value depends on the interactionof the input variables. If None, then nodes are expanded untilall leaves are pure or until all leaves contain less thanmin_samples_split samples.If int, values must be in the range `[1, inf)`.",3
,"min_impurity_decrease min_impurity_decrease: float, default=0.0A node will be split if this split induces a decrease of the impuritygreater than or equal to this value.Values must be in the range `[0.0, inf)`.The weighted impurity decrease equation is the following:: N_t / N * (impurity - N_t_R / N_t * right_impurity - N_t_L / N_t * left_impurity)where ``N`` is the total number of samples, ``N_t`` is the number ofsamples at the current node, ``N_t_L`` is the number of samples in theleft c

In [350]:
my_gbtr_preds = my_gbtr.predict(test_data.drop(columns=[target_col]))
sklearn_preds = sklearn_gbtr.predict(test_data.drop(columns=[target_col]))

pd.DataFrame({"Mean Squared Error": [mean_squared_error(test_data[target_col], my_gbtr_preds), mean_squared_error(test_data[target_col], sklearn_preds)],
              "Root Mean Squared Error": [root_mean_squared_error(test_data[target_col], my_gbtr_preds), root_mean_squared_error(test_data[target_col], sklearn_preds)]}, index=["My GBTR", "SKlearn GBTR"])

,Mean Squared Error,Root Mean Squared Error
My GBTR,0.471486,0.235743
SKlearn GBTR,0.473888,0.236944
